In [1]:
from jax import config
config.update('jax_enable_x64', True)

from flax import nnx
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt

from gpjax import Dataset
from gpjax.gps import Prior as GPPrior
from gpjax.kernels.approximations import RFF
from gpjax.kernels import RBF
from gpjax.parameters import transform

from uncprop.utils.gpjax_models import construct_gp, train_gp_hyperpars
from uncprop.core.surrogate import GPJaxSurrogate
from uncprop.utils.gpjax_multioutput import (
    BatchIndependentGP,
    get_batch_gp_from_template,
    fit_batch_independent_gp,
    _make_batched_loss_and_grad,
    _get_single_output_dataset,
)

/Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# latent functions
f1 = lambda x: jnp.sin(x) + 0.3 * jnp.cos(10*x)
f2 = lambda x: -0.5 * x**2

# design data
x = jnp.linspace(0, 10, 5).reshape(-1, 1)
xgrid = jnp.linspace(0, 10, 100).reshape(-1, 1)

y = jnp.hstack([f1(x), f2(x)])
ygrid = jnp.hstack([f1(xgrid), f2(xgrid)])

design = Dataset(x, y)
testdata = Dataset(xgrid, ygrid)

In [ ]:
import jax
import optax
import gpjax as gpx
from gpjax.parameters import DEFAULT_BIJECTION

opt = optax.adam(1e-1)
objective = lambda p, d: -gpx.objectives.conjugate_mll(p, d)

def gp_factory(dataset):
    return construct_gp(dataset, set_bounds=False)[0]

batchgp = get_batch_gp_from_template(gp_factory, design)

In [ ]:
# Baseline for testing: computing loss/gradient one at a time (not vectorized)

def _print_tree_vals(param):
    l = param.prior.kernel.lengthscale.get_value()
    v = param.prior.kernel.variance.get_value()
    sd = param.likelihood.obs_stddev.get_value()
    m = param.prior.mean_function.constant.get_value()

    print(l, v, sd, m)


def single_losses_grads(i):
    D = _get_single_output_dataset(design, i)
    g, p, s = nnx.split(batchgp.posterior_list[i], gpx.parameters.Parameter, ...)
    phi = transform(p, DEFAULT_BIJECTION, inverse=True)

    def l(phi):
        params = transform(phi, DEFAULT_BIJECTION)
        model = nnx.merge(g, params, s)
        return objective(model, D)

    loss = l(phi)
    gradient = jax.grad(l)(phi)

    print(f'loss: {loss}')
    print('gradient:')
    _print_tree_vals(gradient)

In [ ]:
# vectorized computations

loss_and_grad_vect = _make_batched_loss_and_grad(batchgp, objective, DEFAULT_BIJECTION, design)
def vectorized_losses_grads():
    phi = transform(batchgp.params, DEFAULT_BIJECTION, inverse=True)
    loss, gradient = loss_and_grad_vect(phi)

    print(f'loss: {loss}')
    print('gradient:')
    _print_tree_vals(gradient)

In [ ]:
print('GP 1:')
single_losses_grads(0)

print('\nGP 2:')
single_losses_grads(1)

print('\nVectorized:')
vectorized_losses_grads()

In [ ]:
phi = transform(batchgp.params, DEFAULT_BIJECTION, inverse=True)
print(f'Starting loss: {loss_and_grad_vect(phi)[0]}')

new_batchgp, history = fit_batch_independent_gp(
    batch_gp=batchgp,
    objective=objective,
    optim=opt,
    num_iters=1000
)

phi_final = transform(new_batchgp.params, DEFAULT_BIJECTION, inverse=True)
print(f'Final loss: {loss_and_grad_vect(phi_final)[0]}')

In [ ]:
xnew = jnp.array([3, 5, 6, 8]).reshape(-1, 1)
gp = new_batchgp.batch_posterior
ker = gp.prior.kernel
meanf = gp.prior.mean_function

surrogate = GPJaxSurrogate(gp=new_batchgp.batch_posterior,
                           design=design)

In [ ]:
def plot_gps(design, testdata, gp: GPJaxSurrogate, pred=None):
    fig, axs = plt.subplots(1, gp.output_dim)
    if gp.output_dim > 1:
        axs = axs.ravel()
    else:
        axs = [axs]

    xgrid = testdata.X.ravel()
    if pred is None:
        pred = gp(testdata.X)
    
    for i, ax in enumerate(axs):
        m = jnp.atleast_2d(pred.mean)[i]
        sd = jnp.atleast_2d(pred.stdev)[i]

        ax.fill_between(xgrid, m-2*sd, m+2*sd, alpha=0.4, color='lightblue')
        ax.plot(xgrid, testdata.y[:,i], color='black', label='true')    
        ax.plot(xgrid, m, color='blue', label='mean')
        ax.plot(design.X.ravel(), design.y[:,i], 'ro')
        ax.legend()

    fig.tight_layout()
    return fig, axs

In [ ]:
plot_gps(design, testdata, surrogate)

In [ ]:
# single output case still works as expected
idx = 0
design_single = _get_single_output_dataset(design, idx)
testdata_single = _get_single_output_dataset(testdata, idx)

single_gp, bijection = construct_gp(design_single, set_bounds=False)
single_gp, _ = train_gp_hyperpars(single_gp, bijection, design_single)
single_surr = GPJaxSurrogate(single_gp, design_single)

fig, ax = plot_gps(design_single, testdata_single, single_surr)


In [ ]:
# conditioning methods currently only work for single-output
xnew = jnp.array([[6.0]])
ynew = f1(xnew)
newdata = Dataset(xnew, ynew)
conditional_pred = single_surr.condition_then_predict(testdata_single.X, given=(xnew, ynew))

plot_gps(design_single + newdata, testdata_single, single_surr, pred=conditional_pred)


### Validating batch vs single output predictions

In [ ]:
def test_batch_gp(surrogate, batchgp, train_data, test_inputs):
    test_batch_gp_hyperpars(surrogate, batchgp)
    test_batch_gp_meanf(surrogate, batchgp, test_inputs)
    test_batch_gp_gram(surrogate, batchgp, test_inputs)
    test_batch_gp_cross_cov(surrogate, batchgp, test_inputs)
    # test_batch_gp_pred(surrogate, batchgp, train_data, test_inputs)

def test_batch_gp_hyperpars(surrogate, batchgp):
    posts = batchgp.posterior_list

    # hyperparameters
    l_surr = surrogate.gp.prior.kernel.lengthscale.get_value()
    l_posts = jnp.array([post.prior.kernel.lengthscale.get_value() for post in posts])

    v_surr = surrogate.gp.prior.kernel.variance
    v_posts = jnp.array([post.prior.kernel.variance.get_value() for post in posts])

    obs_sd_surr = surrogate.gp.likelihood.obs_stddev.get_value()
    obs_sd_posts = jnp.array([post.likelihood.obs_stddev.get_value() for post in posts])

    m_surr = surrogate.gp.prior.mean_function.constant.get_value()
    m_posts = jnp.array([post.prior.mean_function.constant.get_value() for post in posts])

    print('lengthscales equal:', jnp.array_equal(l_surr, l_posts))
    print('variances equal:', jnp.array_equal(v_surr, v_posts))
    print('obs stdevs equal:', jnp.array_equal(obs_sd_surr, obs_sd_posts))
    print('obs stdevs equal:', jnp.array_equal(m_surr, m_posts))


def test_batch_gp_meanf(surrogate, batchgp, test_inputs):
    post_list = batchgp.posterior_list
    mean_surr = surrogate.gp.prior.mean_function(test_inputs)
    mean_posts = jnp.hstack([post.prior.mean_function(test_inputs) for post in post_list])
    print(f'prior means equal: {jnp.array_equal(mean_surr, mean_posts)}')

def test_batch_gp_gram(surrogate, batchgp, test_inputs):
    post_list = batchgp.posterior_list
    gram_surr = surrogate.prior_gram(test_inputs)
    gram_posts = jnp.stack([post.prior.kernel.gram(test_inputs).to_dense() for post in post_list])
    print(f'prior grams equal: {jnp.array_equal(gram_surr, gram_posts)}')

def test_batch_gp_cross_cov(surrogate, batchgp, test_inputs):
    post_list = batchgp.posterior_list
    cross_surr = surrogate.prior_cross_covariance(test_inputs, test_inputs)
    cross_posts = jnp.stack([post.prior.kernel.cross_covariance(test_inputs, test_inputs) for post in post_list])
    print(f'prior cross covs equal: {jnp.array_equal(cross_surr, cross_posts)}')

def test_batch_gp_pred(surrogate, batchgp, train_data, test_inputs):
    post_list = batchgp.posterior_list
    pred_surr = surrogate(test_inputs)
    surr_mean = pred_surr.mean
    surr_var = pred_surr.variance

    for i in range(batchgp.dim_out):
        pred_post_latent = post_list[i](test_inputs, train_data)
        pred_post = post_list[i].likelihood(pred_post_latent)

        print(f'output {i+1} predictions:')
        print(f'\tpred means equal: {jnp.array_equal(surr_mean[i], pred_post.mean)}')
        print(f'\tpred variances equal: {jnp.array_equal(surr_var[i], pred_post.variance)}')

In [ ]:
# fit and train 
batchgp = get_batch_gp_from_template(gp_factory, design)

batchgp, history = fit_batch_independent_gp(
    batch_gp=batchgp,
    objective=objective,
    optim=opt,
    num_iters=1000
)
surrogate = GPJaxSurrogate(gp=batchgp.batch_posterior,
                           design=design)

 # run tests
test_batch_gp(surrogate, batchgp, surrogate.design, testdata.X)

In [ ]:
idx = 1

single_design = Dataset(surrogate.design.X, surrogate.design.y[:,[idx]])
pred = surrogate(testdata.X)
pred_baseline_latent = batchgp.posterior_list[idx](testdata.X, single_design)
pred_baseline = batchgp.posterior_list[idx].likelihood(pred_baseline_latent)

fig, axs = plt.subplots(1, 2)
axs = axs.ravel()
ax = axs[0]
ax.scatter(pred_baseline.mean, pred.mean[idx])
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
grid = jnp.linspace(xmin, xmax, 100)
ax.plot(grid, grid, color='gray', linewidth=1, linestyle='--', label='y = x')
ax.set_title('mean')

ax = axs[1]
ax.scatter(pred_baseline.variance, pred.variance[idx])
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
grid = jnp.linspace(xmin, xmax, 100)
ax.plot(grid, grid, color='gray', linewidth=1, linestyle='--', label='y = x')
ax.set_title('variance')

fig.show()

In [ ]:
U = testdata.X[:2]

idx = 1
gram_surr = surrogate.prior_gram(U)
gram2 = surrogate.gp.prior.kernel.gram(U).to_dense()
gram_baseline = batchgp.posterior_list[idx].prior.kernel.gram(U).to_dense()

In [ ]:
print(gram_surr[idx]) # equal
print(gram2[..., idx]) # equal
print(gram_baseline)

In [ ]:
a = batchgp.posterior_list[0].prior.mean_function(U)
b = batchgp.posterior_list[1].prior.mean_function(U)

jnp.hstack([a, b])

In [ ]:
def __call__(self, x: Float[Array, " D"], y: Float[Array, " D"]) -> ScalarFloat:
    x = self.slice_input(x) / self.lengthscale.value 
    y = self.slice_input(y) / self.lengthscale.value
    K = self.variance.value * jnp.exp(-0.5 * squared_distance(x, y))
    return K.squeeze()

In [ ]:
u = U[0]
v = U[1]

kernel = surrogate.gp.prior.kernel

kernel(u, v)

In [ ]:
batchgp.posterior_list[1].prior.kernel(u, v)

# NEW

In [2]:
from gpjax.kernels.stationary import RBF

from uncprop.utils.gpjax_multioutput import (
    BatchDenseKernelComputation,
    BatchedStationaryKernel,
    BatchedRBF,
)

In [ ]:
def setup_case_one(key):
    d, q = 1, 3
    x = jnp.array([1, 2, 3])
    y = jnp.array([4, 5])
    z = jnp.array(6)

    lengthscales = jnp.array([1, 2, 3])[:, None]
    variances = jnp.array([1, 2, 3])

    return d, q, x, y, z, lengthscales, variances

def setup_case_two(key):
    d, q = 2, 2
    kx, ky, kz = jr.split(key, 3)

    x = jr.uniform(kx, (3, 2))
    y = jr.uniform(ky, (2, 2))
    z = jr.uniform(kz, (2,))

    lengthscales = jnp.array([[0.5, 0.1], [0.4, 0.3]]) # (q, d)
    variances = 1.0

    return d, q, x, y, z, lengthscales, variances

def kernel_list_to_batch(kers, x, y):
    return jnp.stack([ker.cross_covariance(x, y) for ker in kers])

def test_kernel(key, test_fn):
    d, q, x, y, z, l, v = test_fn(key)
    kernel = BatchedRBF(batch_dim=q, input_dim=d, lengthscale=l, variance=v)

    x = x.reshape(-1, d)
    y = y.reshape(-1, d)
    z = z.reshape(-1, d)

    l = kernel.lengthscale
    v = kernel.variance

    kers = [RBF(lengthscale=l[i], variance=v[i], n_dims=d) for i in range(q)]

    def k0(x, y):
        return kernel_list_to_batch(kers, x, y)

    def k1(x, y):
        return kernel.cross_covariance(x, y)

    assert jnp.array_equal(k0(x, y), k1(x, y))

In [62]:
# test batched kernel
d, q, x, y, z, lengthscales, variances = setup_case_one(_)

kernel = BatchedRBF(batch_dim=q,
                    input_dim=d,
                    lengthscale=lengthscales, 
                    variance=variances)

In [63]:
key = jr.key(325234)
test_keys = jr.split(key, 2)

test_kernel(test_keys[0], setup_case_one)
test_kernel(test_keys[1], setup_case_two)